**COVID WEB SCRAPING**

In [20]:
!apt install unixodbc-dev
!pip install pyodbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unixodbc-dev is already the newest version (2.3.9-5).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [21]:
import pandas as pd
#import geopandas as gpd
import requests
from bs4 import BeautifulSoup
import re
from datetime import date
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pyodbc
from datetime import datetime
import plotly.graph_objects as go

**Scraping new Data from www.worldometers.info**

In [22]:
r = requests.get("https://www.worldometers.info/coronavirus/country/us/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c = r.content
soup=BeautifulSoup(c,"html.parser")

#print(soup.prettify)
x1 = soup.title.string
s = ''.join(x for x in x1 if x.isdigit())
today = date.today()

In [23]:
z = print(today)

2023-12-18


**Listing all the cases reported and Deaths occured till today**

In [24]:
cases = s[0:6]
print('Cases Reported: as of ',today,' is ',cases)
deaths = s[7:13]
print('Deaths Reported: as of ',today,' is ',deaths)
table = soup.find('table')
table_rows = table.find_all('tr')
List = []

Cases Reported: as of  2023-12-18  is  
Deaths Reported: as of  2023-12-18  is  


**Loooping to find the state Wise Division of cases**


In [25]:
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    #print(row)
    List.append(row)
df = pd.DataFrame(List)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,,USA Total,"109,879,377",,"1,189,378",,"107,702,524","987,475","332,047","3,594","1,186,546,440","3,585,652",,,
2,1,\nCalifornia,"12,543,800",\n,"\n 106,...",,"\n12,400,039","\n 37,5...","317,466","2,687","\n 202,...","5,132,573","\n 39,5...",\n[1],\n ...
3,2,\nTexas,"8,937,117",\n,"\n 95,5...",,"\n8,812,097","\n 29,5...","308,220","3,294","\n 73,9...","2,548,789","\n 28,9...",\n[1],\n ...
4,3,\nFlorida,"7,874,754",\n,"\n 92,5...",,"\n7,762,777","\n 19,4...","366,647","4,308","\n 66,8...","3,110,202","\n 21,4...",\n[1],\n ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,60,\n Fede...,"71,485",\n,\n 324 ...,,\n N/A ...,\n N/A ...,,,"\n 129,...",,\n,\n[1],\n
62,61,\n Gran...,122,\n,\n 7 ...,,\n 115 ...,\n 0 ...,,,\n,,\n,\n[1],\n
63,62,\n Wuha...,3,\n,\n,,\n 3 ...,\n 0 ...,,,\n 3\n ...,,\n,\n,\n
64,63,\n Diam...,46,\n,\n,,\n 46 ...,\n 0 ...,,,\n 46\n...,,\n,\n,\n


**Arranging the data in the columnar format**

In [26]:
df = pd.DataFrame(List)
df = df.iloc[1:,0:6]
df.columns = ['State', 'Total_Cases', 'New_Cases', 'Total_Deaths', 'New_Deaths', 'Active_Cases']
df['State'] = df['State'].str[1:]
df['New_Cases'] = df['New_Cases'].str[1:]
df['Total_Deaths'] = df['Total_Deaths'].str[1:]
df['Active_Cases'] = df['Active_Cases'].str[1:]
df

,State,Total_Cases,New_Cases,Total_Deaths,New_Deaths,Active_Cases
1,,USA Total,"09,879,377",,"1,189,378",
2,,\nCalifornia,"2,543,800",,"\n 106,...",
3,,\nTexas,",937,117",,"\n 95,5...",
4,,\nFlorida,",874,754",,"\n 92,5...",
5,,\nNew York,",323,821",,"\n 82,3...",
...,...,...,...,...,...,...
61,0,\n Fede...,"1,485",,\n 324 ...,
62,1,\n Gran...,22,,\n 7 ...,
63,2,\n Wuha...,,,\n,
64,3,\n Diam...,6,,\n,


**Cleaning the data**

In [27]:
def remove(string):
    return string.replace(" ", "")

df['New_Cases'] = df['New_Cases'].str.replace('+', '')
df['New_Cases'] = df['New_Cases'].str.replace(',','')
remove(df['New_Cases'])

df['New_Deaths'] = df['New_Deaths'].str.replace('+', '')
df['New_Deaths'] = df['New_Deaths'].str.replace(',', '')
df['New_Deaths'] = df['New_Deaths'].str.strip()
remove(df['New_Deaths'])

df['Total_Cases'] = df['Total_Cases'].str.replace(',', '')
df['Total_Cases'] = df['Total_Cases'].str.replace('+', '')
remove(df['Total_Cases'])

df['Total_Cases'] = df['Total_Cases'].str.replace('\n', '')
remove(df['Total_Cases'])

df['Total_Deaths'] = df['Total_Deaths'].str.replace(',', '')
df['Total_Deaths'] = df['Total_Deaths'].str.replace('+', '')
remove(df['Total_Deaths'])

df['Active_Cases'] = df['Active_Cases'].str.replace(',', '')
df['Active_Cases'] = df['Active_Cases'].str.replace('+', '')
remove(df['Active_Cases'] )

df.fillna(0)
#df.replace('',0)

df

<ipython-input-27-9e31c9c40f56>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['New_Cases'] = df['New_Cases'].str.replace('+', '')
<ipython-input-27-9e31c9c40f56>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['New_Deaths'] = df['New_Deaths'].str.replace('+', '')
<ipython-input-27-9e31c9c40f56>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Total_Cases'] = df['Total_Cases'].str.replace('+', '')
<ipython-input-27-9e31c9c40f56>:21: FutureWarning: The default value of regex will change from True to False in

,State,Total_Cases,New_Cases,Total_Deaths,New_Deaths,Active_Cases
1,,USA Total,09879377,,1189378,
2,,California,2543800,,106183,
3,,Texas,937117,,95512,
4,,Florida,874754,,92520,
5,,New York,323821,,82367,
...,...,...,...,...,...,...
61,0,Federa...,1485,,324,
62,1,Grand ...,22,,7,
63,2,Wuhan ...,,,,
64,3,Diamon...,6,,,
